In [ ]:
import requests
from io import StringIO
import pandas as pd
import sqlite3
import logging
from typing import Dict, Tuple

In [ ]:
data = {
  'hired_employees': {
      'url': 'https://raw.githubusercontent.com/deherinu/GB_Test/main/data/hired_employees.csv',
      'schema': {
          'id': 'int64',
          'name': 'object',
          'datetime': 'object',
          'department_id': 'float64',
          'job_id': 'float64',
      },
  },
  'jobs': {
      'url': 'https://raw.githubusercontent.com/deherinu/GB_Test/main/data/jobs.csv',
      'schema': {
          'id': 'int64',
          'job': 'object',
      },
  },
  'departments': {
      'url': 'https://raw.githubusercontent.com/deherinu/GB_Test/main/data/departments.csv',
      'schema': {
          'id': 'int64',
          'department': 'object',
      },
  },
  }

In [ ]:
def create_db(db_file: str) -> Tuple[sqlite3.Connection, sqlite3.Cursor]:
  """
  Create a SQLite database and return a connection and cursor.
  """
  conn = sqlite3.connect(db_file)
  cursor = conn.cursor()
  return conn, cursor

In [ ]:
def create_tables(conn: sqlite3.Connection, cursor: sqlite3.Cursor) -> None:
  """Create tables in the database if they do not exist."""
  tables = {
      'employees': '''
          CREATE TABLE IF NOT EXISTS employees (
              id INTEGER PRIMARY KEY,
              name TEXT NOT NULL,
              datetime TEXT NOT NULL,
              department_id INTEGER,
              job_id INTEGER
          )
      ''',
      'jobs': '''
          CREATE TABLE IF NOT EXISTS jobs (
              id INTEGER PRIMARY KEY,
              job TEXT NOT NULL
          )
      ''',
      'departments': '''
          CREATE TABLE IF NOT EXISTS departments (
              id INTEGER PRIMARY KEY,
              department TEXT NOT NULL
          )
      '''
  }

  for table_name, sql in tables.items():
      cursor.execute(sql)
      print(f"Table '{table_name}' created or already exists.")

  conn.commit()

In [ ]:
def get_data_to_df(data: str):
  data_url = data.get('url')
  response = requests.get(url = data_url)
  if response.status_code == 200:
    schema = data['schema']
    header_names = schema.keys()
    df = pd.read_csv(StringIO(response.text), names= header_names, header=None, dtype=schema)
    logging.info("Data downloaded successfully")
    return df
  else:
    logging.error(f"Error getting the data: {response.status_code} {response.reason}")

In [ ]:
def read_and_insert_data_by_chunks(conn: sqlite3.Connection, cursor: sqlite3.Cursor, df: pd.DataFrame, table_name: str, chunksize: int) -> None:
  """Insert data into the database in chunks."""
  try:
    df.to_sql(table_name, conn, if_exists='replace', index=False, chunksize=chunksize)
    logging.info(f"Data inserted into table '{table_name}' in chunks.")
  except Exception as e:
    logging.error(f"Error inserting data into table '{table_name}': {e}")
    raise

In [ ]:
def query_data(conn: sqlite3.Connection, cursor: sqlite3.Cursor) -> None:

    '''
      Number of employees hired for each job and department in 2021 divided by
      quarter. The table must be ordered alphabetically by department and job.
    '''
    query1 = '''
    SELECT
        departments.department,
        jobs.job,
        SUM(CASE WHEN strftime('%m', employees.datetime) BETWEEN '01' AND '03' THEN 1 ELSE 0 END) AS Q1,
        SUM(CASE WHEN strftime('%m', employees.datetime) BETWEEN '04' AND '06' THEN 1 ELSE 0 END) AS Q2,
        SUM(CASE WHEN strftime('%m', employees.datetime) BETWEEN '07' AND '09' THEN 1 ELSE 0 END) AS Q3,
        SUM(CASE WHEN strftime('%m', employees.datetime) BETWEEN '10' AND '12' THEN 1 ELSE 0 END) AS Q4
    FROM employees
    JOIN departments ON employees.department_id = departments.id
    JOIN jobs ON employees.job_id = jobs.id
    GROUP BY departments.department, jobs.job
    ORDER BY departments.department, jobs.job
    '''

    '''
      List of ids, name and number of employees hired of each department that hired more
      employees than the mean of employees hired in 2021 for all the departments, ordered
      by the number of employees hired (descending).
    '''
    query2 = '''
    WITH hires AS (
        SELECT
            departments.id,
            departments.department,
            COUNT(employees.id) AS hired_employees
        FROM employees
        JOIN departments ON employees.department_id = departments.id
        WHERE strftime('%Y', employees.datetime) = '2021'
        GROUP BY departments.id, departments.department
    ),
    hires_mean AS (
        SELECT AVG(hired_employees) AS mean_hired_employees
        FROM hires
    )
    SELECT
        hires.id,
        hires.department,
        hires.hired_employees
    FROM hires
    JOIN hires_mean
    ON hires.hired_employees > hires_mean.mean_hired_employees
    ORDER BY hires.hired_employees DESC
    '''

    try:
        cursor.execute(query1)
        results1 = cursor.fetchall()
        print("Query 1 executed successfully.")
        for row in results1:
            print(row)

        cursor.execute(query2)
        results2 = cursor.fetchall()
        print("Query 2 executed successfully.")
        for row in results2:
            print(row)

    except Exception as e:
        logging.error(f"Error executing SQL queries: {e}")
        raise


In [ ]:
def main() -> None:
  try:
    conn, cursor = create_db('globant_test_db')
    create_tables(conn, cursor)
    df_hired_employees = get_data_to_df(data['hired_employees'])
    df_jobs = get_data_to_df(data['jobs'])
    df_departments = get_data_to_df(data['departments'])

    read_and_insert_data_by_chunks(conn, cursor, df_hired_employees, 'employees', 1000)
    read_and_insert_data_by_chunks(conn, cursor, df_jobs, 'jobs', 1000)
    read_and_insert_data_by_chunks(conn, cursor, df_departments, 'departments', 1000)

    query_data(conn, cursor)

  except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")

In [ ]:
if __name__ == "__main__":
  main()